In [62]:
# %matplotlib notebook
# %matplotlib inline
import numpy as np
import pickle
np.random.seed(123)
import collections, copy, pickle
from importlib import reload
from dateutil.parser import parse
import scipy.linalg
import pandas as pd
import sklearn
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
from IPython.display import HTML

In [172]:
from mlxtend.frequent_patterns import apriori

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [64]:
import util.plot
import util.string

In [65]:
# https://github.com/pbloem/machine-learning/blob/master/worksheets/Worksheet%202%2C%20Sklearn.ipynb

In [66]:
# data = pd.read_csv('ODI-2019-clean.csv', sep=';')
fn = 'ODI-2019-clean.pkl'
# load (old) data from disk
with open(fn, 'rb') as f:
    data = pickle.load(f)

In [6]:
# data.head()

## Categorical models

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

# Discretization

https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html#sphx-glr-auto-examples-preprocessing-plot-discretization-strategies-py

Strategies:
-    ‘uniform’: The discretization is uniform in each feature, which means that the bin widths are constant in each dimension.
-    quantile’: The discretization is done on the quantiled values, which means that each bin has approximately the same number of samples.
       - this causes outliers to be grouped together
-    ‘kmeans’: The discretization is based on the centroids of a KMeans clustering procedure.

In [8]:
class Encoders: pass
E = Encoders()
E.discretizers = {}
E.encoders = {}

In [101]:
key = 'Other'
# reload(util.data)
most_common = util.data.select_most_common(data.Programme, n=8, key=key)
value = np.array(list(most_common.values()))
# note that pd.where different than np.where
keys = most_common.keys()
data.Programme.where(data.Programme.isin(keys), key, inplace=True)

In [102]:
def discretize(data, k, n_bins=5):
    X = data[k]
    avg = np.nanmedian([x for x in X])
    X = np.where(np.isnan(X), avg, X)
    X = X.reshape(-1,1)
    bins = np.repeat(n_bins, X.shape[1]) # e.g. [5,3] for 2 features
    # encode to integers
    est = preprocessing.KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='kmeans')
    est.fit(X)
    data[k + ' bin'] = est.transform(X)
    E.discretizers[k] = est
    s = ''
    for st in [round(a,3) for a in est.bin_edges_[0]]:
        if k == 'Year':
            st = int(round(st))
        s += str(st) + ', '
    print('%s: $\\{%s\\}$\n' % (k,s[:-2]))

In [103]:
numerical = ['Year', 'Money', 'Neighbours', 'Stress level']
for k in numerical:
    discretize(data, k)

Year: $\{1971, 1981, 1991, 1994, 1996, 2001\}$

Money: $\{0.0, 13.77, 37.19, 64.316, 88.56, 100.0\}$

Neighbours: $\{0.0, 12.711, 28.4, 42.5, 64.25, 80.0\}$

Stress level: $\{0.0, 15.307, 38.186, 62.413, 86.179, 100.0\}$



In [104]:
# def plot_discretizer(X, enc):
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
    
#     xx, yy = np.meshgrid(
#         np.linspace(X[:, 0].min(), X[:, 0].max(), 300),
#         np.linspace(X[:, 1].min(), X[:, 1].max(), 300))
#     grid = np.c_[xx.ravel(), yy.ravel()]

#     ax.set_xlim(xx.min(), xx.max())
#     ax.set_ylim(yy.min(), yy.max())
#     ax.set_xticks(())
#     ax.set_yticks(())

#     # transform the dataset with KBinsDiscretizer
#     enc.fit(X)
#     grid_encoded = enc.transform(grid)

#     ax = plt.subplot(len(X_list), len(strategies) + 1, i)

#      # horizontal stripes
#     horizontal = grid_encoded[:, 0].reshape(xx.shape)
#     ax.contourf(xx, yy, horizontal, alpha=.5)
#     # vertical stripes
#     vertical = grid_encoded[:, 1].reshape(xx.shape)
#     ax.contourf(xx, yy, vertical, alpha=.5)

#     ax.scatter(X[:, 0], X[:, 1], edgecolors='k')
#     ax.set_xlim(xx.min(), xx.max())
#     ax.set_ylim(yy.min(), yy.max())
#     ax.set_xticks(())
#     ax.set_yticks(())
#     if ds_cnt == 0:
#         ax.set_title("strategy='%s'" % (strategy, ), size=14)

In [132]:
def init_encoder(columns):
    E.encoders['x'] = preprocessing.OneHotEncoder()
    enc = E.encoders['x']
    enc.fit(columns)
    return enc.transform(columns)

categorical = ['ML', 'IR', 'Stat', 'DB', 'Gender', 'Chocolate', 'Stand Up', 'Programme']
y = 'ML'
categorical.remove(y)
keys = [k + ' bin' for k in numerical] + categorical
X_enc = init_encoder(data[keys])
E.encoders['x'].categories_

[array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array(['No', 'Unknown', 'Yes'], dtype=object),
 array(['No', 'Unknown', 'Yes'], dtype=object),
 array(['No', 'Unknown', 'Yes'], dtype=object),
 array(['female', 'male', 'unknown'], dtype=object),
 array(['Fat', 'Neither', 'Slim', 'Unknown'], dtype=object),
 array(['no', 'unknown', 'yes'], dtype=object),
 array(['AI', 'BA', 'CLS', 'CS', 'Other'], dtype=object)]

In [133]:
def init_label_encoder(column):
    E.encoders['y'] = preprocessing.LabelEncoder()
    enc = E.encoders['y']
    enc.fit(column)
    return enc.transform(column)

Y_enc = init_label_encoder(data[y])
E.encoders['y'].classes_

array(['No', 'Yes'], dtype=object)

In [134]:
X_enc.shape, Y_enc.shape

((276, 44), (276,))

In [135]:
x_train, x_test, y_train, y_test = train_test_split(X_enc, Y_enc, test_size=0.5)
x_train.shape, y_train.shape

((138, 44), (138,))

In [176]:
def cross_validation(model_func, x_train, y_train, k=None, results=None, v=0):
    # Train for 5 folds, returing ROC AUC. You can also try 'accuracy' as a scorer
    n_folds = 5
    scores_acc = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring='accuracy') # roc_auc accuracy
    scores_roc = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring='roc_auc') # roc_auc accuracy
    if results is not None:
        results[k] = (scores_acc, scores_roc)
    if v:
        print('scores per fold ', scores_acc)
        print('  mean score    ', np.mean(scores_acc))
        print('  standard dev. ', np.std(scores_acc))

In [182]:
models = {
          'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear',
                                                           multi_class='ovr'),
#           'SGD': sklearn.linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3),
#           'SVC auto': sklearn.svm.SVC(gamma='auto'), 
          'SVC': sklearn.svm.SVC(kernel='linear'), 
#           'SVC polynomial': sklearn.svm.SVC(kernel='poly', gamma='auto', degree=4),    
          'Decision Tree':  sklearn.tree.DecisionTreeClassifier(),
          'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
#           'KNN 10': sklearn.neighbors.KNeighborsClassifier(n_neighbors=10),
          'Ensemble Random Forest': sklearn.ensemble.RandomForestClassifier(n_estimators=100),
#           'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=100)
         }

results = {}
for k,m in models.items():
    print(k)
    cross_validation(m, x_train, y_train, k, results)

Logit
SVC
Decision Tree
KNN 5
Ensemble Random Forest


In [183]:
print('Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\\\ \n\\hline')
best_k = ''
best_mean = 0
for k, (scores_acc, scores_roc) in results.items():
    if np.mean(scores_acc) > best_mean:
        best_mean = np.mean(scores_acc)
        best_k = k
    print('%s & %0.4f & %0.4f & %0.4f & %0.4f\\\\' % (k, np.mean(scores_acc), np.std(scores_acc), np.mean(scores_roc), np.std(scores_roc)))
print('\nbest acc:', best_k, round(best_mean,4))

Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\ 
\hline
Logit & 0.7320 & 0.0586 & 0.7602 & 0.0577\\
SVC & 0.7254 & 0.0833 & 0.7472 & 0.0513\\
Decision Tree & 0.6386 & 0.0675 & 0.6366 & 0.0456\\
KNN 5 & 0.7249 & 0.0825 & 0.7615 & 0.0891\\
Ensemble Random Forest & 0.7611 & 0.0527 & 0.7810 & 0.0704\\

best acc: Ensemble Random Forest 0.7611


In [186]:
print('Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\\\ \n\\hline')
best_k = ''
best_mean = 0
for k, (scores_acc, scores_roc) in results.items():
    if np.mean(scores_roc) > best_mean:
        best_mean = np.mean(scores_roc)
        best_k = k
print('\nbest roc:', best_k, round(best_mean,4))

Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\ 
\hline

best roc: Ensemble Random Forest 0.781


In [185]:
# reinit models
models = {
          'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear',
                                                           multi_class='ovr'),
#           'SGD': sklearn.linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3),
#           'SVC auto': sklearn.svm.SVC(gamma='auto'), 
          'SVC': sklearn.svm.SVC(kernel='linear'), 
#           'SVC polynomial': sklearn.svm.SVC(kernel='poly', gamma='auto', degree=4),    
          'Decision Tree':  sklearn.tree.DecisionTreeClassifier(),
          'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
#           'KNN 10': sklearn.neighbors.KNeighborsClassifier(n_neighbors=10),
          'Ensemble Random Forest': sklearn.ensemble.RandomForestClassifier(n_estimators=100),
#           'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=100)
         }

# train best model on whole dataset
model = models[best_k]
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

for v in [sklearn.metrics.accuracy_score(y_test, y_pred), 
          sklearn.metrics.roc_auc_score(y_test, y_pred)]:
    print(round(v,4))

0.6449
0.618


# Association rules

In [50]:
subkeys = []
for i,k in enumerate(keys):
    for v in E.encoders['x'].categories_[i]:
        subkeys.append(k + '_' + str(v))

assert len(subkeys) == pd.DataFrame(X_enc.toarray()).shape[1]

In [51]:
# data_enc = pd.DataFrame(X_enc.toarray(), columns=subkeys, dtype=bool)
data_enc = pd.SparseDataFrame(X_enc, columns=subkeys, default_fill_value=False)
data_enc.head()

,Year bin_0.0,Year bin_1.0,Year bin_2.0,Year bin_3.0,Year bin_4.0,Money bin_0.0,Money bin_1.0,Money bin_2.0,Money bin_3.0,Money bin_4.0,...,Chocolate_Unknown,Stand Up_no,Stand Up_unknown,Stand Up_yes,Programme_AI,Programme_BA,Programme_CLS,Programme_CS,Programme_Other,Programme_QRM
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [52]:
# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
frequent_itemsets = apriori(data_enc, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.967391,(Neighbours bin_0.0),1
1,0.619565,(ML_Yes),1
2,0.876812,(Stat_Yes),1
3,0.659420,(Gender_male),1
4,0.902174,(Stand Up_no),1
5,0.605072,"(ML_Yes, Neighbours bin_0.0)",2
6,0.855072,"(Stat_Yes, Neighbours bin_0.0)",2
7,0.637681,"(Gender_male, Neighbours bin_0.0)",2
8,0.880435,"(Stand Up_no, Neighbours bin_0.0)",2
9,0.804348,"(Stat_Yes, Stand Up_no)",2


In [53]:
frequent_itemsets[ (frequent_itemsets['length'] >= 3) &
                   (frequent_itemsets['support'] >= 0.6) ]

,support,itemsets,length
10,0.789855,"(Stat_Yes, Stand Up_no, Neighbours bin_0.0)",3


In [54]:
frequent_itemsets[ (frequent_itemsets['length'] >= 2) &
                   (frequent_itemsets['support'] >= 0.7) ]

,support,itemsets,length
6,0.855072,"(Stat_Yes, Neighbours bin_0.0)",2
8,0.880435,"(Stand Up_no, Neighbours bin_0.0)",2
9,0.804348,"(Stat_Yes, Stand Up_no)",2
10,0.789855,"(Stat_Yes, Stand Up_no, Neighbours bin_0.0)",3


In [ ]:
def cmap_discretize(cmap, N):
    """Return a discrete colormap from the continuous colormap cmap.
    
        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.
        
        https://scipy-cookbook.readthedocs.io/items/Matplotlib_ColormapTransformations.html
    """
    if type(cmap) == str:
        cmap = plt.get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N+1)
    cdict = {}
    for ki, key in enumerate(('red','green','blue')):
        cdict[key] = [(indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki]) for i in range(N+1)]
    # Return colormap object.
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)

In [ ]:
def e(i,k):
    # unit vector i of length k
    return np.eye(k)[i]

def fix_labels(labels):
    # update in-place
    for i, v in enumerate(labels.copy()):
        if v is None:
            labels[i] = 'None'
        else:
            labels[i] = fix_label(labels[i])

            
def fix_label(x):
    max_length = 12
    x = str.title(str(x))
    if x is None:
        assert False
        return 'None'
    translations = {'Ja': 'Yes', 'Nee': 'No', '1': 'Yes', '0':'No',
                   'I have no idea what you are talking about': 'Unknown'}
    if x in translations.keys():
        return translations[x]
    return x[:max_length]

def length_exceeds(labels, t=10) -> bool:
    return np.any([len(label) > t for label in labels])

# def plot_classifier_helper(data, xlabels, ylabel, clf):
def plot_classifier(data, xlabels, ylabel, clf):
    rcParams['font.size'] = 14
    X_enc = init_encoder(data[xlabels])
    Y_enc = init_label_encoder(data[ylabel])

#     Y_enc += np.random.randint(0,3, Y_enc.size)
    clf.fit(X_enc, Y_enc)

    categories = E.encoders['x'].categories_
    dims = tuple([c.size for c in categories]) # n features
    n_dims = sum(dims)
    # n combinations = x * y                   ### x^2 * y^2
    X = np.empty((np.prod([x**2 for x in dims]), 1))
    
    # X = np.empty((np.prod(dims), n_dims))
    X = np.empty((dims[0], dims[1], n_dims))
    for i,j in np.ndindex(X.shape[:2]):
        X[i,j] = np.concatenate([e(i,dims[0]), e(j, dims[1])])

    X_enc = X.reshape((dims[0] * dims[1], n_dims))
    Y_enc = clf.predict(X_enc)
    Y_enc[0] = 0
    Y_enc[1] = 1
    Y_enc[2] = 2
    # Y = E.encoders['y'].inverse_transform(Y_enc)
    Y = Y_enc.reshape(dims)

    # def plot_classifier()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    # color bar labels correspond to Y (unlike x or y)
    cb_labels = E.encoders['y'].classes_
    cb_ticks = E.encoders['y'].transform(cb_labels)
    n = cb_labels.size
    c = cmap_discretize('rainbow', n) # terrain bone plasma rainbow pink cubehelix jet
    plt.imshow(Y.T, origin='lower', cmap=c)
    
    x_labels = categories[0]
    y_labels = categories[1]
    fix_labels(x_labels)
    fix_labels(y_labels)
    n_x = len(x_labels)
    n_y = len(y_labels)    
    if length_exceeds(x_labels, 5) or n_x > 5:
        rotation = 45
    else:
        rotation = 0        
#         plt.subplots_adjust(bottom=0.15)
    plt.xticks(np.arange(n_x), x_labels, rotation=rotation)
    if length_exceeds(y_labels, 5) or n_y > 5:
        rotation = 45
    else:
        rotation = 0
    plt.yticks(np.arange(n_y), y_labels, rotation=rotation)

    util.plot.show_grid(ax, Y)
    #  colorbar
    cb = plt.colorbar()
    cb.set_ticks(cb_ticks)
    cb.set_ticklabels(cb_labels)
#     cb.ax.set_yticklabels(cb_labels)

#     ticks = np.arange(5)
#     labels = E.encoders['y'].inverse_transform(np.linspace(y_min, y_max, 5))

# def show_grid(ax, Y):
#     # clear grid
#     ax.grid(False)        
#     # disable spines
#     for edge, spine in ax.spines.items():
#             spine.set_visible(False)
#     # add custom grid
#     # add white grid to distinguish cells
#     print(Y.shape)
#     lw = 3
#     ax.set_xticks(np.arange(Y.shape[0]+1) - 0.5 + 0.005*lw, minor=True)
#     ax.set_yticks(np.arange(Y.shape[1]+1) - 0.5 + 0.01*lw, minor=True)
#     ax.grid(which="minor", color='w', linestyle='-', linewidth=lw) # 3
#     ax.tick_params(which="minor", bottom=False, left=False)    

In [ ]:
np.random.seed(42)
# clf = SVC(gamma='auto')
clf = DecisionTreeClassifier()
plot_classifier(data, ['Stat', 'Year bin'], 'Gender', clf)
plt.suptitle('Effect of Stat, Year on prediction of Gender')

In [ ]:
plot_classifier(data, ['Programme', 'Year bin'], 'Gender', clf)

In [ ]:
# The number of divisions of the cmap we have
k = 10

# Random test data
A = np.random.random((10,10))*k
c = cmap_discretize('jet', k)

# First show without
plt.subplot(121)
plt.imshow(A,interpolation='nearest',cmap=c)
plt.colorbar()

# Now label properly
plt.subplot(122)
plt.imshow(A,interpolation='nearest',cmap=c)

cb = plt.colorbar()
labels = np.arange(0,k,1)
loc    = labels + .5
cb.set_ticks(loc)
cb.set_ticklabels(labels)

In [ ]:
X = np.ndindex(())

In [ ]:
x = np.arange(-5, 5, 0.1)
y = np.arange(-5, 5, 0.1)
xx, yy = np.meshgrid(x, y, sparse=True)
z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)
h = plt.contourf(x,y,z)

In [ ]:
x = np.arange(-5, 5, 0.1)
y = np.arange(-5, 5, 0.1)
xx, yy = np.meshgrid(x, y, sparse=True)
z = np.sin(xx + 0*yy)
h = plt.contourf(x,y,z)
x.shape, y.shape, z.shape

In [ ]:
# xx

In [ ]:
# from mlxtend.plotting import plot_decision_regions
# import matplotlib.pyplot as plt
# from sklearn import datasets
# from sklearn.svm import SVC

# # Loading some example data
# iris = datasets.load_iris()
# X = iris.data[:, [0, 2]]
# y = iris.target

# # Training a classifier
# svm = SVC(C=0.5, kernel='linear')
# svm.fit(x_train, y_train)


# # Plotting decision regions
# # plot_decision_regions(x_train.toarray(), y_train, clf=svm, legend=2)

# enc = E.encoders['x']
# enc.

# # Decision region for feature 3 = 1.5
# value = 0
# # Plot training sample with feature 3 = 1.5 +/- 0.75
# width = 0
# plot_decision_regions(X, y, clf=svm,
#                       filler_feature_values={2: value},
#                       filler_feature_ranges={2: width},
#                       legend=2, ax=ax)


# # Adding axes annotations
# plt.xlabel('sepal length [cm]')
# plt.ylabel('petal length [cm]')
# plt.title('SVM on Iris')
# plt.show()


In [ ]:
# from mlxtend.plotting import plot_decision_regions

# n = 10
# plot_decision_regions(x_test[:n], y_test.astype(np.integer)[:n], 
#                       clf=clf, res=0.1);

In [ ]:
# from sklearn.metrics import roc_curve, auc

# # The linear classifier doesn't produce class probabilities by default. We'll retrain it for probabilities.
# linear = SVC(kernel='linear', probability=True)
# linear.fit(x_train, y_train)

# # We'll need class probabilities from each of the classifiers
# y_linear = linear.predict_proba(x_test)
# y_tree  = tree.predict_proba(x_test)
# y_knn   = knn.predict_proba(x_test)

# # Compute the points on the curve
# # We pass the probability of the second class (KIA) as the y_score
# curve_linear = sklearn.metrics.roc_curve(y_test, y_linear[:, 1])
# curve_tree   = sklearn.metrics.roc_curve(y_test, y_tree[:, 1])
# curve_knn    = sklearn.metrics.roc_curve(y_test, y_knn[:, 1])

# # Compute Area Under the Curve
# auc_linear = auc(curve_linear[0], curve_linear[1])
# auc_tree   = auc(curve_tree[0], curve_tree[1])
# auc_knn    = auc(curve_knn[0], curve_knn[1])

# plt.plot(curve_linear[0], curve_linear[1], label='linear (area = %0.2f)' % auc_linear)
# plt.plot(curve_tree[0], curve_tree[1], label='tree (area = %0.2f)' % auc_tree)
# plt.plot(curve_knn[0], curve_knn[1], label='knn (area = %0.2f)'% auc_knn)

# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC curve');

# plt.legend();